<a href="https://colab.research.google.com/github/Nebil1/UNDP-FTL-AI/blob/main/Task_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Deep Learning Lab 4: plastics classification through computer vision

### Import libraries

In [ ]:
import tensorflow as tf
from google.colab import drive
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers, models
import matplotlib.pyplot as plt
import numpy as np
from sklearn.metrics import confusion_matrix

### Mount Google Drive

In [ ]:
drive.mount('/content/drive')
base_dir = "/content/drive/MyDrive/Dataset_plastics_mhadlekar_UN"

Mounted at /content/drive


### Preprocess Images
  - Validation split: 75% training, 25% validation

In [ ]:
batch_size = 32

train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    width_shift_range=0.2,
    height_shift_range=0.2,
    fill_mode='nearest',
    validation_split=0.25
)

val_datagen = ImageDataGenerator(rescale=1./255, validation_split=0.25)

### Train/Val Generators

In [ ]:
train_set = train_datagen.flow_from_directory(
    base_dir,
    target_size=(256, 256),
    batch_size=batch_size,
    class_mode='categorical',
    subset='training'
)

val_set = val_datagen.flow_from_directory(
    base_dir,
    target_size=(256, 256),
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation'
)

train_num = train_set.samples
val_num = val_set.samples

Found 436 images belonging to 4 classes.
Found 143 images belonging to 4 classes.


### Load VGG16 Model (Pretrained)
  - using transfer learning to benefit from pre-trained weights from ImageNet. Only the last block (block5) is fine-tuned to adapt to plastic classification.

In [ ]:
from keras.applications.vgg16 import VGG16
from keras.models import Model
from keras.layers import Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.optimizers import SGD

in_shape = (256, 256, 3)
out_shape = 4

vgg_model = VGG16(include_top=False, input_shape=in_shape)
for layer in vgg_model.layers:
    layer.trainable = False  # freeze all layers

# Fine-tune last block
for layer_name in ['block5_conv1', 'block5_conv2', 'block5_conv3', 'block5_pool']:
    vgg_model.get_layer(layer_name).trainable = True

58889256/58889256 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


### Custom Classifier Head
  - stacking a fully connected classifier on top of the convolutional base to perform multi-class classification.

In [ ]:
flat1 = Flatten()(vgg_model.output)
fcon1 = Dense(4096, activation='relu', kernel_initializer='he_uniform')(flat1)
fdrop1 = Dropout(0.25)(fcon1)
fbn1 = BatchNormalization()(fdrop1)

fcon2 = Dense(4096, activation='relu', kernel_initializer='he_uniform')(fbn1)
fdrop2 = Dropout(0.25)(fcon2)
fbn2 = BatchNormalization()(fdrop2)

output = Dense(out_shape, activation='softmax')(fbn2)
model = Model(inputs=vgg_model.inputs, outputs=output)

### Compile the Model

In [ ]:
opt = SGD(learning_rate=0.01, momentum=0.9)
model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 256, 256, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_conv1 (Conv2D)           │ (None, 256, 256, 64)   │         1,792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_conv2 (Conv2D)           │ (None, 256, 256, 64)   │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_pool (MaxPooling2D)      │ (None, 128, 128, 64)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_conv1 (Conv2D)           │ (None, 128, 128, 128)  │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_conv2 (Conv2D)           │ (None, 128, 128, 128)  │       147,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_pool (MaxPooling2D)      │ (None, 64, 64, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv1 (Conv2D)           │ (None, 64, 64, 256)    │       295,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv2 (Conv2D)           │ (None, 64, 64, 256)    │       590,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv3 (Conv2D)           │ (None, 64, 64, 256)    │       590,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_pool (MaxPooling2D)      │ (None, 32, 32, 256)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv1 (Conv2D)           │ (None, 32, 32, 512)    │     1,180,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv2 (Conv2D)           │ (None, 32, 32, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv3 (Conv2D)           │ (None, 32, 32, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_pool (MaxPooling2D)      │ (None, 16, 16, 512)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv1 (Conv2D)           │ (None, 16, 16, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv2 (Conv2D)           │ (None, 16, 16, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv3 (Conv2D)           │ (None, 16, 16, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_pool (MaxPooling2D)      │ (None, 8, 8, 512)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 32768)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 4096)           │   134,221,824 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 4096)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 4096)           │        16,384 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 4096)           │    16,781,312 │
├─────────────────────────────────┼────────────────────────┼─────────────

 Total params: 165,766,980 (632.35 MB)

 Trainable params: 158,115,332 (603.16 MB)

 Non-trainable params: 7,651,648 (29.19 MB)

### Save Checkpoints
  - saves the best model automatically when validation accuracy improves during training

In [ ]:
from keras.callbacks import ModelCheckpoint

weightpath = "VGG_Best.weights.h5"
checkpoint = ModelCheckpoint(
    weightpath,
    monitor='val_accuracy',
    verbose=1,
    save_best_only=True,
    save_weights_only=True
)
callbacks_list = [checkpoint]

### Train the Model
  - steps_per_epoch = samples // batch_size standardizes training

In [ ]:
history = model.fit(
    train_set,
    steps_per_epoch=train_num // batch_size,
    validation_data=val_set,
    validation_steps=val_num // batch_size,
    epochs=30,
    callbacks=callbacks_list
)

/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/60


/usr/local/lib/python3.11/dist-packages/keras/src/models/functional.py:237: UserWarning: The structure of `inputs` doesn't match the expected structure.
Expected: ['keras_tensor']
Received: inputs=Tensor(shape=(None, 256, 256, 3))
  warnings.warn(msg)


13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 33s/step - accuracy: 0.2603 - loss: 1.9813 
Epoch 1: val_accuracy improved from -inf to 0.26562, saving model to VGG_Best.weights.h5
13/13 ━━━━━━━━━━━━━━━━━━━━ 575s 44s/step - accuracy: 0.2631 - loss: 1.9872 - val_accuracy: 0.2656 - val_loss: 106.6609
Epoch 2/60
 1/13 ━━━━━━━━━━━━━━━━━━━━ 6:41 33s/step - accuracy: 0.3438 - loss: 2.4568

/usr/local/lib/python3.11/dist-packages/keras/src/trainers/epoch_iterator.py:107: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self._interrupted_warning()



Epoch 2: val_accuracy did not improve from 0.26562
13/13 ━━━━━━━━━━━━━━━━━━━━ 186s 13s/step - accuracy: 0.3438 - loss: 2.4568 - val_accuracy: 0.1016 - val_loss: 577.4150
Epoch 3/60
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 34s/step - accuracy: 0.4170 - loss: 2.5961 
Epoch 3: val_accuracy did not improve from 0.26562
13/13 ━━━━━━━━━━━━━━━━━━━━ 562s 44s/step - accuracy: 0.4175 - loss: 2.6082 - val_accuracy: 0.1016 - val_loss: 5337.7246
Epoch 4/60
 1/13 ━━━━━━━━━━━━━━━━━━━━ 6:35 33s/step - accuracy: 0.3438 - loss: 3.5019
Epoch 4: val_accuracy did not improve from 0.26562
13/13 ━━━━━━━━━━━━━━━━━━━━ 139s 9s/step - accuracy: 0.3438 - loss: 3.5019 - val_accuracy: 0.0781 - val_loss: 696.3372
Epoch 5/60
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 34s/step - accuracy: 0.4898 - loss: 3.6916 
Epoch 5: val_accuracy did not improve from 0.26562
13/13 ━━━━━━━━━━━━━━━━━━━━ 561s 44s/step - accuracy: 0.4915 - loss: 3.6995 - val_accuracy: 0.1875 - val_loss: 908.3592
Epoch 6/60
 1/13 ━━━━━━━━━━━━━━━━━━━━ 6:54 35s/step - accuracy:

### Save Final Model
  - Saves the entire model (architecture + weights), so that it is re-usable later without retraining.

In [ ]:
filepath = "VGGModel_final.h5"
model.save(filepath)

### Plot Accuracy and Loss

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

sns.set()
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(1, len(loss) + 1)

# Accuracy
plt.plot(epochs, acc, label='Training Accuracy', color='green')
plt.plot(epochs, val_acc, label='Validation Accuracy', color='blue')
plt.title('Training vs Validation Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

# Loss
plt.plot(epochs, loss, label='Training Loss', color='red')
plt.plot(epochs, val_loss, label='Validation Loss', color='orange')
plt.title('Training vs Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.show()